In [1]:
import requests, zipfile, io
r = requests.get("http://files.grouplens.org/datasets/movielens/ml-20m.zip")
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [24]:
z.filelist


[<ZipInfo filename='ml-20m/' filemode='drwxr-xr-x' external_attr=0x10>,
 <ZipInfo filename='ml-20m/genome-scores.csv' compress_type=deflate filemode='-rw-r--r--' file_size=323544381 compress_size=60442683>,
 <ZipInfo filename='ml-20m/genome-tags.csv' compress_type=deflate filemode='-rw-r--r--' file_size=18103 compress_size=8359>,
 <ZipInfo filename='ml-20m/links.csv' compress_type=deflate filemode='-rw-r--r--' file_size=570090 compress_size=246387>,
 <ZipInfo filename='ml-20m/movies.csv' compress_type=deflate filemode='-rw-r--r--' file_size=1397542 compress_size=499415>,
 <ZipInfo filename='ml-20m/ratings.csv' compress_type=deflate filemode='-rw-r--r--' file_size=533444411 compress_size=132712092>,
 <ZipInfo filename='ml-20m/README.txt' compress_type=deflate filemode='-rw-r--r--' file_size=10261 compress_size=3940>,
 <ZipInfo filename='ml-20m/tags.csv' compress_type=deflate filemode='-rw-r--r--' file_size=16603996 compress_size=4787884>]

In [2]:
import pandas as pd


Read the dataset using pandas.


In [3]:

movies = pd.read_csv('./ml-20m/movies.csv', sep=',')
tags = pd.read_csv('./ml-20m/tags.csv', sep=',',parse_dates=['timestamp'])
ratings = pd.read_csv('./ml-20m/ratings.csv', nrows=100000, sep=',',parse_dates=['timestamp'])

Extract the first row from tags and print its type.


In [4]:
row_0 = tags.loc[0]
type(row_0)
print(row_0)

userId                18
movieId             4141
tag          Mark Waters
timestamp     1240597180
Name: 0, dtype: object


Extract row 0, 11, 2000 from tags DataFrame.


In [5]:
tags.iloc[ [0,11,2000] ]

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,1240597180
11,65,1783,noir thriller,1368149983
2000,910,68554,conspiracy theory,1368043943


Print index, columns of the DataFrame.


In [6]:
tags.index

RangeIndex(start=0, stop=465564, step=1)

In [7]:
tags.columns


Index(['userId', 'movieId', 'tag', 'timestamp'], dtype='object')

Calculate descriptive statistics for the 'ratings' column of the ratings DataFrame. Verify using describe().


In [8]:
ratings['rating'].describe()


count    100000.000000
mean          3.507605
std           1.062928
min           0.500000
25%           3.000000
50%           3.500000
75%           4.000000
max           5.000000
Name: rating, dtype: float64

Filter out ratings with rating > 5


In [9]:
filter_rat = ratings['rating'] > 5
print(filter_rat)

0        False
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10       False
11       False
12       False
13       False
14       False
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
99970    False
99971    False
99972    False
99973    False
99974    False
99975    False
99976    False
99977    False
99978    False
99979    False
99980    False
99981    False
99982    False
99983    False
99984    False
99985    False
99986    False
99987    False
99988    False
99989    False
99990    False
99991    False
99992    False
99993    False
99994    False
99995    False
99996    False
99997    False
99998    False
99999    False
Name: rating, Length: 100000, dtype: bool


7.Find how many null values, missing values are present. Deal with them. Print out how many rows have been modified.



In [10]:
tags.isnull().any()


userId       False
movieId      False
tag           True
timestamp    False
dtype: bool

In [23]:
tags.shape

(465564, 5)

8.Filter out movies from the movies DataFrame that are of type 'Animation'.



In [13]:
movies[['title','genres']].head()


,title,genres
0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,Jumanji (1995),Adventure|Children|Fantasy
2,Grumpier Old Men (1995),Comedy|Romance
3,Waiting to Exhale (1995),Comedy|Drama|Romance
4,Father of the Bride Part II (1995),Comedy


In [14]:
is_animation = movies['genres'].str.contains('Animation')

print(movies['title'].count())
movies[is_animation].head(15)

27278


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
12,13,Balto (1995),Adventure|Animation|Children
47,48,Pocahontas (1995),Animation|Children|Drama|Musical|Romance
236,239,"Goofy Movie, A (1995)",Animation|Children|Comedy|Romance
241,244,Gumby: The Movie (1995),Animation|Children
310,313,"Swan Princess, The (1994)",Animation|Children
360,364,"Lion King, The (1994)",Adventure|Animation|Children|Drama|Musical|IMAX
388,392,"Secret Adventures of Tom Thumb, The (1993)",Adventure|Animation
547,551,"Nightmare Before Christmas, The (1993)",Animation|Children|Fantasy|Musical
553,558,"Pagemaster, The (1994)",Action|Adventure|Animation|Children|Fantasy


Find the average rating of movies.


In [15]:
average_rating = ratings[['movieId','rating']].groupby('movieId').mean()
average_rating.head()

,rating
movieId,
1,3.968880
2,3.250000
3,3.230159
4,3.384615
5,3.253623


Perform an inner join of movies and tags based on movieId.


In [16]:
joindf = movies.join(tags, on='movieId', how='inner', lsuffix='_left', rsuffix='_right')
joindf.head()

,movieId,movieId_left,title,genres,userId,movieId_right,tag,timestamp
0,1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,65,208,dark hero,1368150078
1,2,2,Jumanji (1995),Adventure|Children|Fantasy,65,353,dark hero,1368150079
2,3,3,Grumpier Old Men (1995),Comedy|Romance,65,521,noir thriller,1368149983
3,4,4,Waiting to Exhale (1995),Comedy|Drama|Romance,65,592,dark hero,1368150078
4,5,5,Father of the Bride Part II (1995),Comedy,65,668,bollywood,1368149876


Print out the 5 movies that belong to the Comedy genre and have rating greater than 4.


In [17]:
box_office = movies.merge(average_rating, on='movieId', how='inner')
is_highly_rated = box_office['rating'] >= 4.0
is_comedy = box_office['genres'].str.contains('Comedy')
box_office[is_comedy & is_highly_rated][:5]

,movieId,title,genres,rating
77,82,Antonia's Line (Antonia) (1995),Comedy|Drama,4.090909
99,106,Nobody Loves Me (Keiner liebt mich) (1994),Comedy|Drama,4.000000
114,125,Flirting With Disaster (1996),Comedy,4.176471
170,189,Reckless (1995),Comedy|Fantasy,4.000000
200,223,Clerks (1994),Comedy,4.008264


Split 'genres' into multiple columns.


In [18]:
movie_genres = movies['genres'].str.split('|', expand=True)
movie_genres

,0,1,2,3,4,5,6,7,8,9
0,Adventure,Animation,Children,Comedy,Fantasy,None,None,None,None,None
1,Adventure,Children,Fantasy,None,None,None,None,None,None,None
2,Comedy,Romance,None,None,None,None,None,None,None,None
3,Comedy,Drama,Romance,None,None,None,None,None,None,None
4,Comedy,None,None,None,None,None,None,None,None,None
5,Action,Crime,Thriller,None,None,None,None,None,None,None
6,Comedy,Romance,None,None,None,None,None,None,None,None
7,Adventure,Children,None,None,None,None,None,None,None,None
8,Action,None,None,None,None,None,None,None,None,None
9,Action,Adventure,Thriller,None,None,None,None,None,None,None


Extract year from title e.g. (1995).


In [20]:
movies['year'] = movies['title'].str.extract('.*\((.*)\).*', expand=True)
print(movies['year'].head())

0    1995
1    1995
2    1995
3    1995
4    1995
Name: year, dtype: object


Select rows based on timestamps later than 2015-02-01.


In [21]:

tags['parsed_time'] = pd.to_datetime(tags['timestamp'], unit='s')
greater_than_t = tags['parsed_time'] > '2015-02-01'
selected_rows = tags[greater_than_t]
print(tags.shape)
print(selected_rows.shape)

(465564, 5)
(12130, 5)


Sort the tags DataFrame based on timestamp.


In [22]:
tags.sort_values(by='parsed_time', ascending=True)[:10]


,userId,movieId,tag,timestamp,parsed_time
333932,100371,2788,monty python,1135429210,2005-12-24 13:00:10
333927,100371,1732,coen brothers,1135429236,2005-12-24 13:00:36
333924,100371,1206,stanley kubrick,1135429248,2005-12-24 13:00:48
333923,100371,1193,jack nicholson,1135429371,2005-12-24 13:02:51
333939,100371,5004,peter sellers,1135429399,2005-12-24 13:03:19
333922,100371,47,morgan freeman,1135429412,2005-12-24 13:03:32
333921,100371,47,brad pitt,1135429412,2005-12-24 13:03:32
333936,100371,4011,brad pitt,1135429431,2005-12-24 13:03:51
333937,100371,4011,guy ritchie,1135429431,2005-12-24 13:03:51
333920,100371,32,bruce willis,1135429442,2005-12-24 13:04:02
